In [1]:
%matplotlib inline
import os,sys
pwd = os.getcwd()
parent = '/'.join(pwd.split('/')[:-1])
sys.path.insert(0,parent)
os.chdir(parent)
#%%
print('-'*30)
print(os.getcwd())
print('-'*30)
#%%
import torch
import torch.optim as optim
import torch.nn as nn
import pandas as pd
from core.DAZLE import DAZLE
from core.CUBDataLoader_standard_split import CUBDataLoader
from core.helper_func import eval_zs_gzsl,visualize_attention,eval_zs_gzsl#,get_attribute_attention_stats
from global_setting import NFS_path
#from core.Scheduler import Scheduler
import importlib
import pdb
import numpy as np

------------------------------
/home/project_amadeus/home/hbdat/[RELEASE]_DenseAttentionZSL
------------------------------


In [2]:
idx_GPU = 0
device = torch.device("cuda:{}".format(idx_GPU) if torch.cuda.is_available() else "cpu")

In [3]:
torch.backends.cudnn.benchmark = True

In [4]:
dataloader = CUBDataLoader(NFS_path,device,is_unsupervised_attr=False,is_balance=False)

!!!!!!!!!! Standard Split !!!!!!!!!!
/home/project_amadeus/mnt/raptor/hbdat/Attention_over_attention/
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
CUB
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
_____
/home/project_amadeus/mnt/raptor/hbdat/Attention_over_attention/data/CUB/feature_map_ResNet_101_CUB.hdf5
Expert Attr
Finish loading data in  61.513001


In [5]:
def get_attr_entropy(att):  #the lower the more discriminative it is
    eps = 1e-8
    mass=np.sum(att,axis = 0,keepdims=True)
    att_n = np.divide(att,mass+eps)
    entropy = np.sum(-att_n*np.log(att_n+eps),axis=0)
    assert len(entropy.shape)==1
    return entropy

In [6]:
batch_size = 50
nepoches = 1
niters = dataloader.ntrain * nepoches//batch_size
dim_f = 2048
dim_v = 300
init_w2v_att = dataloader.w2v_att
att = dataloader.att#dataloader.normalize_att#
normalize_att = dataloader.att
#%% attribute selection
attr_entropy = get_attr_entropy(att.cpu().numpy())
idx_attr_dis = np.argsort(attr_entropy)

In [7]:
print(init_w2v_att.shape)

torch.Size([312, 300])


In [8]:
def get_lr(optimizer):
    lr = []
    for param_group in optimizer.param_groups:
        lr.append(param_group['lr'])
    return lr

In [ ]:
seed = 214#215#
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

batch_size = 50
nepoches = 20
niters = dataloader.ntrain * nepoches//batch_size
dim_f = 2048
dim_v = 300
init_w2v_att = dataloader.w2v_att
att = dataloader.att#dataloader.normalize_att#
normalize_att = dataloader.normalize_att
#assert (att.min().item() == 0 and att.max().item() == 1)

trainable_w2v = True
lambda_ = 0.1
bias = 0
prob_prune = 0
uniform_att_1 = False
uniform_att_2 = False

seenclass = dataloader.seenclasses
unseenclass = dataloader.unseenclasses
desired_mass = 1#unseenclass.size(0)/(seenclass.size(0)+unseenclass.size(0))
report_interval = niters//nepoches#10000//batch_size#

model = DAZLE(dim_f,dim_v,init_w2v_att,att,normalize_att,
            seenclass,unseenclass,
            lambda_,
            trainable_w2v,normalize_V=False,normalize_F=True,is_conservative=True,
            uniform_att_1=uniform_att_1,uniform_att_2=uniform_att_2,
            prob_prune=prob_prune,desired_mass=desired_mass, is_conv=False,
            is_bias=True)
model.to(device)

setup = {'pmp':{'init_lambda':0.1,'final_lambda':0.1,'phase':0.8},
         'desired_mass':{'init_lambda':-1,'final_lambda':-1,'phase':0.8}}
print(setup)
#scheduler = Scheduler(model,niters,batch_size,report_interval,setup)

params_to_update = []
params_names = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        params_names.append(name)
        print("\t",name)
#%%
lr = 0.0001
weight_decay = 0.00005#0.000#0.#
momentum = 0.9#0.#
#%%
lr_seperator = 1
lr_factor = 1
print('default lr {} {}x lr {}'.format(params_names[:lr_seperator],lr_factor,params_names[lr_seperator:]))
optimizer  = optim.RMSprop( params_to_update ,lr=lr,weight_decay=weight_decay, momentum=momentum)
print('-'*30)
print('learing rate {}'.format(lr))
print('trainable V {}'.format(trainable_w2v))
print('lambda_ {}'.format(lambda_))
print('optimized seen only')
print('optimizer: RMSProp with momentum = {} and weight_decay = {}'.format(momentum,weight_decay))
print('-'*30)

In [10]:
best_performance = [0,0,0,0]
for i in range(0,niters):
    model.train()
    optimizer.zero_grad()
    
    batch_label, batch_feature, batch_att = dataloader.next_batch(batch_size)
    out_package = model(batch_feature)
    
    in_package = out_package
    in_package['batch_label'] = batch_label
    
    out_package=model.compute_loss(in_package)
    loss,loss_CE,loss_cal = out_package['loss'],out_package['loss_CE'],out_package['loss_cal']
    
    loss.backward()
    optimizer.step()
    if i%report_interval==0:
        print('-'*30)
        acc_seen, acc_novel, H, acc_zs = eval_zs_gzsl(dataloader,model,device,bias_seen=-bias,bias_unseen=bias)
        
        if acc_zs > best_performance[3]:
            best_performance = [acc_seen, acc_novel, H, acc_zs]
        stats_package = {'iter':i, 'loss':loss.item(), 'loss_CE':loss_CE.item(),
                         'loss_cal': loss_cal.item(),
                         'acc_seen':best_performance[0], 'acc_novel':best_performance[1], 'H':best_performance[2], 'acc_zs':best_performance[3]}
        
        print(stats_package)

------------------------------
bias_seen 0 bias_unseen 0
{'iter': 0, 'loss': 5.307318687438965, 'loss_CE': 5.271770000457764, 'loss_cal': 0.35548919439315796, 'acc_seen': nan, 'acc_novel': 0.040221136063337326, 'H': 0, 'acc_zs': 0.04022114351391792}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 177, 'loss': 1.6891649961471558, 'loss_CE': 1.5842840671539307, 'loss_cal': 1.0488090515136719, 'acc_seen': nan, 'acc_novel': 0.4655452370643616, 'H': 0, 'acc_zs': 0.5405555367469788}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 354, 'loss': 1.5217909812927246, 'loss_CE': 1.4060150384902954, 'loss_cal': 1.1577597856521606, 'acc_seen': nan, 'acc_novel': 0.510532796382904, 'H': 0, 'acc_zs': 0.6040189266204834}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 531, 'loss': 1.156381607055664, 'loss_CE': 1.0383371114730835, 'loss_cal': 1.1804451942443848, 'acc_seen': nan, 'acc_novel': 0.5140025019645691, 'H': 0, 'acc_zs': 0.6206724643707275}
--